## Zadanie 1 (2 pkt.)

Napisz funkcję rozstrzygającą czy dana lista jest palindromem, ale taką, która wykonuje co najwyżej ⌊n/2⌋ wywołań rekurencyjnych, gdzie n jest nieznaną długością listy (nie potrzeba i nie wolno jej liczyć). Funkcja nie powinna również alokować nowych list.

In [6]:
let is_palindrome l =
    let rec aux = function
        | (l,[]) -> (true,l)
        | (l,[x]) -> (true, List.tl l)
        | (x::xs, y::y'::ys) -> let p = aux (xs,ys) in
            let isp = fst p and list = snd p in
                if isp == false
                then (false,[])
                else if List.hd list == x 
                    then (true, List.tl list)
                    else (false, [])
    in
        fst @@ aux (l,l)
        
let is_palindrome' l =
    let rec aux = function
        | (l',[]) -> (true,l')
        | (l',[_]) -> (true, List.tl l')
        | (x::xs, _::_::ys) -> 
            let p = aux (xs,ys) in
            let isp = fst p and list = snd p in
                if List.hd list == x && isp == true
                then (true, List.tl list)
                else (false,[])
    in
        fst @@ aux (l,l)
            

File "[6]", line 2, characters 18-387:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a value that is not matched:


val is_palindrome : 'a list -> bool = <fun>


([], _::_::_)
File "[6]", line 16, characters 18-339:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a value that is not matched:
([], _::_::_)


val is_palindrome' : 'a list -> bool = <fun>


In [7]:
is_palindrome' [];;
is_palindrome' [1];;
is_palindrome' [1;2];;
is_palindrome' [1;1];;
is_palindrome' [1;2;1];;
is_palindrome' [1;2;2];;

- : bool = true


- : bool = true


- : bool = false


- : bool = true


- : bool = true


- : bool = false


## Zadanie 2 (2 pkt.)

Rozważmy typ danych dla drzew binarnych, zdefiniowany następująco:

`type 'a btree = Leaf | Node of 'a btree * 'a * 'a btree`

Mówimy, że drzewo jest zbalansowane, jeśli dla każdego węzła v liczby węzłów w lewym i prawym poddrzewie zakorzenionym w v różnią się co najwyżej o 1.

1. Napisz efektywną funkcję sprawdzającą czy dane drzewo jest zbalansowane.
2. Napisz funkcję, która dla zadanej listy etykiet tworzy zbalansowane drzewo, dla którego tę listę można otrzymać przechodząc je w porządku preorder.

In [44]:
type 'a btree = Leaf | Node of 'a btree * 'a * 'a btree

let is_tree_balanced t = 
    let rec aux = function
        | Leaf        -> (true, 1)
        | Node(l,_,r) -> let a = aux l and b = aux r in
            if abs (snd a - snd b) > 1
            then (false,  max (snd a) (snd b))
            else (fst a && fst b, 1 + max (snd a) (snd b)) in
    fst @@ aux t

let rec tree_to_list_VLR = function 
    | Leaf -> []
    | Node(left,e,right) -> e :: ( (tree_to_list_VLR left) @ (tree_to_list_VLR right) )

let halves l = 
    let rec aux acc = function
        | (r, []) -> (List.rev acc,r)
        | (r, [_]) -> (List.rev acc,r)
        | (a::r, _::_::q) -> aux (a::acc) (r,q)
    in
        aux [] (l,l)

let rec list_to_tree = function
    | [] -> Leaf
    | [x] -> Node(Leaf,x,Leaf)
    | x::xs -> let p = halves xs in
        let left = fst p and right = snd p in
        Node(list_to_tree left,x,list_to_tree right)

type 'a btree = Leaf | Node of 'a btree * 'a * 'a btree


val is_tree_balanced : 'a btree -> bool = <fun>


val tree_to_list_VLR : 'a btree -> 'a list = <fun>


File "[44]", line 17, characters 22-155:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a value that is not matched:
([], _::_::_)


val halves : 'a list -> 'a list * 'a list = <fun>


val list_to_tree : 'a list -> 'a btree = <fun>


In [9]:
let testList = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;21];;
let tree = list_to_tree testList;;

is_tree_balanced tree;;
tree_to_list_VLR tree

val testList : int list =
  [1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13; 14; 15; 16; 17; 18; 19; 20; 21]


val tree : int btree =
  Node
   (Node
     (Node (Node (Leaf, 4, Leaf), 3, Node (Leaf, 5, Node (Leaf, 6, Leaf))),
     2,
     Node (Node (Leaf, 8, Node (Leaf, 9, Leaf)), 7,
      Node (Leaf, 10, Node (Leaf, 11, Leaf)))),
   1,
   Node
    (Node (Node (Leaf, 14, Leaf), 13, Node (Leaf, 15, Node (Leaf, 16, Leaf))),
    12,
    Node (Node (Leaf, 18, Node (Leaf, 19, Leaf)), 17,
     Node (Leaf, 20, Node (Leaf, 21, Leaf)))))


- : bool = true


- : int list =
[1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12; 13; 14; 15; 16; 17; 18; 19; 20; 21]


## Zadanie 3 (2 pkt.)

Napisz możliwie efektywne funkcje przejścia wszerz oraz przejścia w głąb (preorder) dla obu reprezentacji drzew wielokierunkowych podanych na wykładzie (zadanie 5 z listy kontrolnej do wykładu 4).

In [10]:
type 'a mtree = MNode of 'a * 'a forest
and 'a forest = EmptyForest | Forest of 'a mtree * 'a forest;;
    
type 'a mtree_lst = MTree of 'a * ('a mtree_lst) list;;

type 'a mtree = MNode of 'a * 'a forest
and 'a forest = EmptyForest | Forest of 'a mtree * 'a forest


type 'a mtree_lst = MTree of 'a * 'a mtree_lst list


In [11]:
let rec preorder_MTree = function
    | MTree(v,l) -> v :: List.fold_left (fun acc t -> acc @ preorder_MTree t) [] l

let rec fold_map_forest f = function
    | EmptyForest -> []
    | Forest(t,f') -> f t @ fold_map_forest f f'

let rec preorder_MNode = function
    | MNode(v,f) -> v :: fold_map_forest preorder_MNode f

let bfs_MTree t =  
    let rec aux = function
        | [] -> []
        | MTree(v,l) :: ls -> v :: aux (ls @ l) 
    in
        aux [t]

let bfs_MNode t =
    let rec aux = function
        | [] -> []
        | MNode(v,f) :: ls -> v :: aux (ls @ fold_map_forest (fun a -> [a]) f)
    in
        aux [t]

val preorder_MTree : 'a mtree_lst -> 'a list = <fun>


val fold_map_forest : ('a mtree -> 'b list) -> 'a forest -> 'b list = <fun>


val preorder_MNode : 'a mtree -> 'a list = <fun>


val bfs_MTree : 'a mtree_lst -> 'a list = <fun>


val bfs_MNode : 'a mtree -> 'a list = <fun>


In [12]:
let test_MTree = MTree(1,[  MTree(2,[ MTree(3,[]) ; MTree(4,[])   ]) ; MTree(5,[]) ;MTree(6,[]) ]);;
let test_MNode = 
MNode(1,Forest(MNode(2, Forest(MNode(3,EmptyForest),Forest(MNode(4,EmptyForest),EmptyForest))), Forest(MNode(5,EmptyForest), Forest(MNode(6,EmptyForest),EmptyForest) )  ));;

preorder_MTree test_MTree;;
preorder_MNode test_MNode;;
bfs_MTree test_MTree;;
bfs_MNode test_MNode;;

val test_MTree : int mtree_lst =
  MTree (1,
   [MTree (2, [MTree (3, []); MTree (4, [])]); MTree (5, []); MTree (6, [])])


val test_MNode : int mtree =
  MNode (1,
   Forest
    (MNode (2,
      Forest (MNode (3, EmptyForest),
       Forest (MNode (4, EmptyForest), EmptyForest))),
    Forest (MNode (5, EmptyForest),
     Forest (MNode (6, EmptyForest), EmptyForest))))


- : int list = [1; 2; 3; 4; 5; 6]


- : int list = [1; 2; 3; 4; 5; 6]


- : int list = [1; 2; 5; 6; 3; 4]


- : int list = [1; 2; 5; 6; 3; 4]


## Zadanie 4 (6 pkt.)

Zdefiniuj typ służący do reprezentacji formuł rachunku zdań składających się ze zmiennych zdaniowych, negacji, koniunkcji i alternatywy.

1. Napisz funkcję sprawdzającą, czy dana formuła jest tautologią. W tym celu należy generować kolejne wartościowania zmiennych zdaniowych występujących w danej formule i sprawdzać dla nich wartość formuły. W przypadku, gdy formuła nie jest tautologią, funkcja powinna, oprócz tej informacji, podać jedno z wartościowań, dla których formuła nie jest prawdziwa.
2. Napisz funkcję, która przekształca zadaną formułę w formułę jej równoważną w negacyjnej postaci normalnej, tj. w której negacja występuje tylko przy zmiennych zdaniowych.
3. Napisz funkcję, która przekształca zadaną formułę w formułę jej równoważną w koniunkcyjnej postaci normalnej.
4. Napisz funkcję sprawdzającą (syntaktycznie), czy zadana formuła jest tautologią, korzystając z faktu, że każdą formułę można przedstawić w koniunkcyjnej postaci normalnej.
5. Napisz funkcję, która przekształca zadaną formułę w formułę jej równoważną w dyzjunkcyjnej postaci normalnej.
6. Napisz funkcję sprawdzającą (syntaktycznie), czy zadana formuła jest sprzeczna, korzystając z faktu, że każdą formułę można przedstawić w dyzjunkcyjnej postaci normalnej.

In [13]:
type 'a formula =    
    | Var of 'a 
    | And of 'a formula * 'a formula 
    | Or of 'a formula * 'a formula 
    | Not of 'a formula

type 'a formula =
    Var of 'a
  | And of 'a formula * 'a formula
  | Or of 'a formula * 'a formula
  | Not of 'a formula


In [14]:
module SS = Set.Make(String);;

module SS :
  sig
    type elt = String.t
    type t = Set.Make(String).t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val max_elt : t -> elt
    val choose : t -> elt
    val split : elt -> t -> t * bool * t
    val find : elt -> t -> elt
    val of_list : elt list -> t
  end


In [15]:
let formula1 = Var("p");;
let formula2 = Or(Not(Var("m")),Var("m"))
let formula3 = And(Or(Var("p"),Var("q")),Var("r"))
let formula4 = Not(And(Var("z"),Var("m")))
let formula5 = And(Or(Not(And(formula4,formula3)) , Not(And(formula4,formula3))) , Not(formula2) )

val formula1 : string formula = Var "p"


val formula2 : string formula = Or (Not (Var "m"), Var "m")


val formula3 : string formula = And (Or (Var "p", Var "q"), Var "r")


val formula4 : string formula = Not (And (Var "z", Var "m"))


val formula5 : string formula =
  And
   (Or
     (Not
       (And (Not (And (Var "z", Var "m")),
         And (Or (Var "p", Var "q"), Var "r"))),
     Not
      (And (Not (And (Var "z", Var "m")),
        And (Or (Var "p", Var "q"), Var "r")))),
   Not (Or (Not (Var "m"), Var "m")))


In [16]:
let rec eval_formula s = function
    | Var(l) -> s l
    | Not(f) -> not @@ eval_formula s f
    | Or(l,r) -> eval_formula s l || eval_formula s r
    | And(l,r) -> eval_formula s l && eval_formula s r

let get_variables f = 
    let rec aux acc_set = function
        | Var(p) -> SS.add p acc_set
        | Not(f) -> aux acc_set f
        | Or(l,r) -> SS.union (aux acc_set l) (aux acc_set r)
        | And(l,r) -> SS.union (aux acc_set l) (aux acc_set r)
    in
        SS.elements @@ aux SS.empty f
        
let add_to_assoc assoc elem = 
    List.fold_left (fun acc l ->  ((elem,true)::l) :: ((elem,false)::l) :: acc ) [] assoc
    
let get_all_evaluations f =
    let vars = get_variables f in
    List.fold_left add_to_assoc [[]] vars
    
let get_eval_function_from_assoc assoc = (function k -> List.assoc k assoc)

let is_tautology f = 
    let evals = get_all_evaluations f in
    let filtered = List.filter 
        (fun a -> fst a == false) 
        (List.map (fun x -> (eval_formula (get_eval_function_from_assoc x) f),x) evals )
    in
        if filtered == []
        then (true,[])
        else (false, snd @@ List.hd filtered)


val eval_formula : ('a -> bool) -> 'a formula -> bool = <fun>


val get_variables : SS.elt formula -> SS.elt list = <fun>


val add_to_assoc : ('a * bool) list list -> 'a -> ('a * bool) list list =
  <fun>


val get_all_evaluations : SS.elt formula -> (SS.elt * bool) list list = <fun>


val get_eval_function_from_assoc : ('a * 'b) list -> 'a -> 'b = <fun>


val is_tautology : SS.elt formula -> bool * (SS.elt * bool) list = <fun>


In [17]:
is_tautology formula1;;
is_tautology formula2;;
is_tautology formula3;;
is_tautology formula4;;
is_tautology formula5;;

- : bool * (SS.elt * bool) list = (false, [("p", false)])


- : bool * (SS.elt * bool) list = (true, [])


- : bool * (SS.elt * bool) list =
(false, [("r", false); ("q", false); ("p", true)])


- : bool * (SS.elt * bool) list = (false, [("z", true); ("m", true)])


- : bool * (SS.elt * bool) list =
(false, [("z", true); ("r", false); ("q", true); ("p", false); ("m", true)])


In [18]:
let rec negation_normal_form = function
    | Not(Not(f)) -> negation_normal_form f
    | Not(Or(l,r)) -> And(negation_normal_form (Not(l)), negation_normal_form (Not(r)))
    | Not(And(l,r)) -> Or(negation_normal_form (Not(l)), negation_normal_form (Not(r)) )
    | Or(l,r) -> Or(negation_normal_form l, negation_normal_form r)
    | And(l,r) -> And(negation_normal_form l, negation_normal_form r)
    | Not(Var(p)) -> Not(Var(p))
    | Var(p) -> Var(p)
(* zamiast ostatnich 2 match'y mógłbym dać rest -> rest, ale w tym wypadku jest cos bedzie nie tak to bedzie 
error zamiast normalnego wykonania funkcji *)

let rec check_negation_normal_form = function
    | Not(Not(_)) -> false
    | Not(Or(_,_)) -> false
    | Not(And(_,_)) -> false
    | Or(l,r) -> check_negation_normal_form l && check_negation_normal_form r
    | And(l,r) -> check_negation_normal_form l && check_negation_normal_form r
    | Not(Var(_)) -> true
    | Var(_) -> true
(* zamiast ostatnich 2 match'y mógłbym dać rest -> rest, ale w tym wypadku jest cos bedzie nie tak to bedzie 
error zamiast normalnego wykonania funkcji *)

val negation_normal_form : 'a formula -> 'a formula = <fun>


val check_negation_normal_form : 'a formula -> bool = <fun>


In [19]:
check_negation_normal_form @@ negation_normal_form formula1;;
check_negation_normal_form @@ negation_normal_form formula2;;
check_negation_normal_form @@ negation_normal_form formula3;;
check_negation_normal_form @@ negation_normal_form formula4;;

- : bool = true


- : bool = true


- : bool = true


- : bool = true


In [21]:
let to_cnf f = 
    let nnf = negation_normal_form f in
    let rec aux = function
        | Or(p,And(q,r)) -> And( aux @@ Or(p,q), aux @@ Or(p,r))
        | Or(And(p,q),r) -> And( aux @@ Or(p,r), aux @@ Or(q,r))
        | And(p,q) -> And( aux p, aux q)
        | rest -> rest
    in
        aux nnf
        

val to_cnf : 'a formula -> 'a formula = <fun>


In [22]:
to_cnf formula1;;
to_cnf formula2;;
to_cnf formula3;;
to_cnf formula4;;
to_cnf formula5;; (* chyba źle *)

- : string formula = Var "p"


- : string formula = Or (Not (Var "m"), Var "m")


- : string formula = And (Or (Var "p", Var "q"), Var "r")


- : string formula = Or (Not (Var "z"), Not (Var "m"))


- : string formula =
And
 (Or
   (Or (And (Var "z", Var "m"),
     Or (And (Not (Var "p"), Not (Var "q")), Not (Var "r"))),
   Or (And (Var "z", Var "m"),
    Or (And (Not (Var "p"), Not (Var "q")), Not (Var "r")))),
 And (Var "m", Not (Var "m")))


In [ ]:
let add_to_set = 
    | Not(Var(p)) -> 

In [129]:
let to_dnf f = 
    let nnf = negate_normal_form f in
    let rec aux = function
        | x -> x
    in
        aux nnf

val to_dnf : 'a formula -> 'a formula = <fun>


## Zadanie 5 (3 pkt.)

Funkcja jest napisana w stylu przekazywania kontynuacji (continuation-passing style, CPS), jeżeli przyjmuje dodatkowy argument funkcyjny zwany kontynuacją, który reprezentuje całą resztę obliczeń jakie mają zostać przeprowadzone po powrocie z tej funkcji. W konsekwencji, funkcje w CPS-ie zwracają wynik wywołując swoją kontynuację, a wszystkie wywołania w programie w CPS-ie są ogonowe. Na przykład, funkcja licząca silnię napisana w CPS-ie wygląda następująco:

`let rec fact_cps n k = if n = 0 then k 1 else fact_cps (n-1) (fun v -> k (n*v))`

Kontynuacja początkowa przekazana funkcji factcps mówi co zrobić z wynikiem obliczenia silni zadanej liczby. Typowe wywołanie funkcji factcps dostaje identyczność jako kontynuację początkową (gdy silnia jest obliczona, wystarczy ją zwrócić):

`let fact n = fact_cps n (fun v -> v)`

Dla drzew binarnych z zadania 2, napisz funkcję `prod : int btree -> int`, która liczy iloczyn wszystkich wartości w drzewie. Zapisz tę funkcję w CPS-ie, a następnie zmodyfikuj otrzymaną funkcję tak by w przypadku napotkania wartości 0 funkcja wykonała bezpośredni skok do miejsca swego wywołania, bez krokowego powracania z rekursji.

In [54]:
(* http://www.cs.cornell.edu/courses/cs3110/2013sp/supplemental/recitations/rec26-cps/rec26.html *)
(* https://sidburn.github.io/blog/2016/05/07/cps-fold *)

let tree_prod t = 
    let rec aux k = function
        | Leaf -> k 1
        | Node(l,e,r) -> aux (fun vl -> aux (fun vr -> k (vl*e*vr)) r ) l
    in
        aux (fun v -> v) t
        
(* chyba tak | jednak nie tak *)
let tree_prod' t = 
    let rec aux k = function
        | Leaf -> k 1
        | Node(l,e,r) -> aux (fun vl -> 
            if vl <> 0
            then aux (fun vr -> 
                if vr <> 0
                then k (vl*e*vr)
                else 0) r 
            else 0) l
    in
        aux (fun v -> if v == 0 then 0 else v) t

val tree_prod : int btree -> int = <fun>


val tree_prod' : int btree -> int = <fun>


In [96]:
let rec threes = 3::threes;;

let rec take k xs = match k with
    | 0 -> []
    | k -> match xs with
           | [] -> failwith "take"
           | y::ys -> y :: (take (k - 1) ys);;

let gen_test_list length = (fun l -> List.map (fun x -> (Random.int x) + 1) @@ take l threes) length;;
let test_list = 0 :: gen_test_list 100000;;
let test_tree = list_to_tree test_list;;
let test_list2 = [1;2;3;4;5;6;7;8;9;10];;
let test_tree2 = list_to_tree test_list2;;

val threes : int list = [3; <cycle>]


val take : int -> 'a list -> 'a list = <fun>


val gen_test_list : int -> int list = <fun>


val test_list : int list =
  [0; 1; 3; 1; 3; 1; 3; 1; 3; 2; 1; 3; 2; 3; 1; 3; 1; 3; 2; 3; 1; 2; 1; 3; 3;
   3; 3; 3; 2; 2; 2; 1; 1; 2; 2; 3; 2; 1; 1; 1; 1; 3; 2; 2; 3; 2; 3; 3; 3; 1;
   2; 2; 2; 2; 3; 3; 3; 3; 1; 2; 2; 2; 1; 2; 2; 2; 3; 2; 3; 1; 1; 1; 1; 2; 1;
   2; 3; 3; 2; 2; 2; 3; 3; 1; 2; 1; 2; 3; 2; 2; 3; 2; 2; 3; 1; 2; 1; 1; 2; 3;
   2; 1; 2; 1; 1; 1; 2; 1; 1; 2; 2; 3; 1; 2; 3; 2; 2; 1; 3; 3; 1; 1; 2; 1; 3;
   3; 2; 2; 3; 1; 3; 3; 3; 2; 2; 2; 1; 1; 2; 2; 2; 3; 3; 3; 1; 2; 2; 3; 2; 3;
   3; 3; 3; 3; 1; 1; 1; 3; 1; 2; 1; 1; 2; 1; 2; 1; 3; 2; 1; 3; 2; 3; 1; 1; 3;
   3; 3; 1; 3; 3; 1; 2; 3; 1; 3; 1; 1; 3; 2; 1; 3; 1; 3; 1; 1; 2; 3; 3; 1; 2;
   2; 2; 3; 3; 3; 3; 3; 3; 2; 1; 2; 3; 2; 2; 1; 1; 1; 1; 2; 2; 1; 1; 1; 2; 1;
   2; 3; 2; 3; 2; 2; 1; 3; 2; 3; 2; 3; 1; 3; 3; 3; 1; 3; 2; 2; 2; 2; 3; 1; 2;
   3; 1; 3; 3; 1; 3; 2; 1; 1; 2; 1; 2; 1; 1; 1; 1; 3; 3; 2; 1; 2; 1; 2; 3; 2;
   2; 1; 2; 2; 1; 1; 2; 3; 3; 1; 2; 2; 1; 1; 1; 1; 1; 2; 2; 1; 2; 1; 2; 2;
   ...]


val test_tree : int btree =
  Node
   (Node
     (Node
       (Node
         (Node
           (Node
             (Node
               (Node
                 (Node
                   (Node
                     (Node
                       (Node
                         (Node
                           (Node
                             (Node (Node (Leaf, 3, Node (Leaf, 1, Leaf)), 1,
                               Node (Leaf, 3, Node (Leaf, 2, Leaf))),
                             3,
                             Node (Node (Leaf, 1, Node (Leaf, 2, Leaf)), 3,
                              Node (Leaf, 1, Node (Leaf, 3, Leaf)))),
                           2,
                           Node
                            (Node (Node (Leaf, 3, Node (Leaf, 3, Leaf)), 3,
                              Node (Leaf, 2, Node (Leaf, 2, Leaf))),
                            3,
                            Node (Node (Leaf, 1, Node (Leaf, 1, Leaf)), 2,
                             Node (Leaf, 2, Node (Leaf

val test_list2 : int list = [1; 2; 3; 4; 5; 6; 7; 8; 9; 10]


val test_tree2 : int btree =
  Node (Node (Node (Leaf, 3, Leaf), 2, Node (Leaf, 4, Node (Leaf, 5, Leaf))),
   1,
   Node (Node (Leaf, 7, Node (Leaf, 8, Leaf)), 6,
    Node (Leaf, 9, Node (Leaf, 10, Leaf))))


In [97]:
is_tree_balanced test_tree;;
tree_prod test_tree2;;
tree_prod' test_tree2;;

- : bool = true


- : int = 3628800


- : int = 3628800


In [99]:
tree_prod test_tree;;

- : int = 0


In [100]:
tree_prod' test_tree;;

- : int = 0


## Zadanie 6 (5 pkt.)

Gdy modyfikujemy strukturę danych, często chcemy wykonać więcej niż jedną operację w pobliskich miejscach. W przypadku czysto funkcyjnych struktur może to skutkować niepotrzebnie długim czasem wykonania, spowodowanym przechodzeniem i odtwarzaniem struktury danych przy każdej zmianie. Aby zapobiec temu problemowi, możemy rozbić operację modyfikacji struktury na dwie części: znajdowania miejsca w strukturze, w którym mamy wykonać zmianę, oraz przeprowadzenia zmiany w konkretnym miejscu; musimy też znaleźć odpowiednią pośrednią reprezentację dla takich miejsc.

1. Zdefiniuj typ danych α place jako pośrednią reprezentację dla typu α list.
2. Zaimplementuj funkcje findNth : α list -> int -> α place oraz collapse : α place -> α list. Pierwsza powinna lokalizować n-te miejsce na liście, druga — zapominać informację o miejscu i zwracać listę na której działamy.
3. Zaimplementuj funkcje add : α -> α place -> α place oraz del : α place -> α place, odpowiednio dodającą i usuwającą element w odpowiednim miejscu listy. Funkcje powinny działać w czasie stałym, oraz spełniać następujące równości:

`collapse (add 3 (findNth [1;2;4] 2)) = [1;2;3;4]
collapse (del (findNth [1;2;4] 2)) = [1;2]
del (add x p) = p dla dowolnych x : α i p : α place`

4. Zaimplementuj funkcje next: α place -> α place oraz prev : α place -> α place, służące do nawigacji w strukturze listy. Te funkcje również powinny działać w czasie stałym.
5. Listy nie są jedynym typem danych dla którego możemy definiować struktury pamiętające miejsce w którym pracujemy. Zdefiniuj typ danych α btplace będący pośrednią reprezentacją pozwalającą na modyfikacje drzew binarnych z Zadania 2 (nie musisz definiować na nim odpowiednich operacji).

In [35]:
(* https://caml.inria.fr/pub/docs/oreilly-book/html/book-ora016.html *)

(* empty list *)
let empty_list = (fun p -> p []);;

(* represents [1;2;3;4;5;6] list *)
let sl = (fun p -> 1::2::3::(p [4;5;6]) );;

(* getting that list *)
let list = sl (fun v -> v);;

(* deleting single element from remembered position *)
(* this representation ofter deleting *)
let sl2 np = sl (fun list -> np (List.tl list));;

(* getting that modified list *)
let list2 = sl2 (fun v -> v);;

(* adding new element 0 to remembered place *)
let sl3 np = sl2 (fun list -> np (0::list));;

(* getting that modigied list *)
let list3 = sl3 (fun v-> v);;

val empty_list : ('a list -> 'b) -> 'b = <fun>


val sl : (int list -> int list) -> int list = <fun>


val list : int list = [1; 2; 3; 4; 5; 6]


val sl2 : (int list -> int list) -> int list = <fun>


val list2 : int list = [1; 2; 3; 5; 6]


val sl3 : (int list -> int list) -> int list = <fun>


val list3 : int list = [1; 2; 3; 0; 5; 6]


In [37]:
type 'a place = Place of (('a list -> 'a list) -> 'a list)

let collapse = function
    | Place(f) -> f (fun v -> v)
    
let del = function
    | Place(f) -> Place( (fun np ->  f (fun l -> np (List.tl list))     ))
    
let add e = function
    | Place(f) -> Place( (fun np ->  f (fun l -> np (e::l))              ))
    
let next = function
    | Place(f) -> Place( (fun np ->  f (fun l -> List.hd l :: np (List.tl l))              ))
    

type 'a place = Place of (('a list -> 'a list) -> 'a list)


val collapse : 'a place -> 'a list = <fun>


val del : int place -> int place = <fun>


val add : 'a -> 'a place -> 'a place = <fun>


val next : 'a place -> 'a place = <fun>


In [34]:
let list_empty = Place((fun p -> p []));;

collapse @@ add 2 @@ add 1 list_empty

val list_empty : 'a place = Place <fun>


- : int list = [2; 1]
